In [1]:
import gensim
import logging

#import string
from nltk.tokenize import RegexpTokenizer # tokenizing
from nltk.corpus import stopwords  # list of stop words
from nltk.stem.wordnet import WordNetLemmatizer # lemmatizer

# Logging code taken from http://rare-technologies.com/word2vec-tutorial/
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Loading model

In [2]:
path = "/Users/stevenfelix/Documents/DataScience_local/Insight/"
file = 'model_full_30M_sg_200_5_2'
model_full = gensim.models.word2vec.Word2Vec.load(path+file)

2018-01-17 10:58:35,293 : INFO : loading Word2Vec object from /Users/stevenfelix/Documents/DataScience_local/Insight/model_full_30M_sg_200_5_2
2018-01-17 10:58:36,096 : INFO : loading wv recursively from /Users/stevenfelix/Documents/DataScience_local/Insight/model_full_30M_sg_200_5_2.wv.* with mmap=None
2018-01-17 10:58:36,097 : INFO : loading syn0 from /Users/stevenfelix/Documents/DataScience_local/Insight/model_full_30M_sg_200_5_2.wv.syn0.npy with mmap=None
2018-01-17 10:58:36,372 : INFO : setting ignored attribute syn0norm to None
2018-01-17 10:58:36,373 : INFO : loading syn1neg from /Users/stevenfelix/Documents/DataScience_local/Insight/model_full_30M_sg_200_5_2.syn1neg.npy with mmap=None
2018-01-17 10:58:36,611 : INFO : setting ignored attribute cum_table to None
2018-01-17 10:58:36,612 : INFO : loaded /Users/stevenfelix/Documents/DataScience_local/Insight/model_full_30M_sg_200_5_2


In [3]:
q="How to find all questions that were duplicates of another question".split()
model_full.predict_output_word(q)

[('that', 0.00012694114),
 ('all', 0.00011346572),
 ('row', 0.0001107741),
 ('one', 0.00010965685),
 ('element', 0.00010931836),
 ('column', 0.00010459485),
 ('match', 0.00010246703),
 ('list', 9.8215613e-05),
 ('number', 9.4541174e-05),
 ('table', 9.423896e-05)]

In [4]:
test_queries=['python binomial regression', 'dropping duplicates by index in pandas dataframe',
             'latent community topic analysis', 'statsmodels mixed effects']

# Prediction algorithm

In [56]:
tokenizer = RegexpTokenizer(r'\w+') # tokens separated by white spice
stops = set(stopwords.words('english')) # list of english stop words
lemma = WordNetLemmatizer()

def clean(title, rmv_stop_words=False):
    tokens = tokenizer.tokenize(title.lower())     # tokenize
    if rmv_stop_words:
        tokens = [i for i in tokens if not i in stops] # remove stop words
    normalized = [lemma.lemmatize(token) for token in tokens] # lemma
    return normalized

def predict_similar(query, rmv_stop_words=False):
    l = []
    q = clean(query, rmv_stop_words=rmv_stop_words)
    print('Original query: {}'.format(query))
    #print(q)
    print('\nTry these related searches:')
    for word in q:
        missing = q[:]
        ind = q.index(word)
        missing.remove(word)
        for syn in model_full.most_similar([word],topn=5):
            full = missing[:]
            full.insert(ind,syn[0])
            l.append(' '.join(full))
    return l

def predict_substitution(query):
    q = clean(query)
    print('Original query: {}'.format(query))
    #print(q)
    print('\nTry these related searches:')
    

In [59]:
preds = predict_similar("loop over pandas dataframe")

Original query: loop over pandas dataframe

Try these related searches:


In [61]:
s = pd.DataFrame(preds)
s

,0
0,looping over panda dataframe
1,iteration over panda dataframe
2,foreach over panda dataframe
3,forloop over panda dataframe
4,iterates over panda dataframe
5,loop through panda dataframe
6,loop throught panda dataframe
7,loop underneath panda dataframe
8,loop keyvaluecollection panda dataframe
9,loop trough panda dataframe


**notes**:
  - 'iteration' substituations work well, possibly because verb?
  - 

In [17]:
model_full.most_similar(['someone'])

[('somebody', 0.859092116355896),
 ('please', 0.7342624068260193),
 ('somone', 0.7300513982772827),
 ('anyone', 0.7129900455474854),
 ('me', 0.6949419975280762),
 ('anybody', 0.6632101535797119),
 ('explain', 0.6601981520652771),
 ('kindly', 0.6287382245063782),
 ('expalin', 0.6191532611846924),
 ('comtimeouts', 0.6093792915344238)]

In [16]:
predict_similar('Someone flagged my question as already answered, but it\'s not', rmv_stop_words=True)

Original query: Someone flagged my question as already answered, but it's not

Try these related searches:


somebody flagged question already answered
please flagged question already answered
somone flagged question already answered
anyone flagged question already answered
me flagged question already answered


someone flagging question already answered
someone abused question already answered
someone marked question already answered
someone tripped question already answered
someone treated question already answered


someone flagged confusion already answered
someone flagged doubt already answered
someone flagged clarification already answered
someone flagged regarding already answered
someone flagged ques already answered


someone flagged question allready answered
someone flagged question been answered
someone flagged question previously answered
someone flagged question aready answered
someone flagged question alread answered


someone flagged question already unanswered
someone 

In [19]:
import pandas as pd

In [21]:
path='/Users/stevenfelix/Downloads/'
file='user-ct-test-collection-01.txt'
data = pd.read_table(path+file, delimiter='\t')
data.iloc[0:100,:]

,AnonID,Query,QueryTime,ItemRank,ClickURL
0,142,rentdirect.com,2006-03-01 07:17:12,NaN,NaN
1,142,www.prescriptionfortime.com,2006-03-12 12:31:06,NaN,NaN
2,142,staple.com,2006-03-17 21:19:29,NaN,NaN
3,142,staple.com,2006-03-17 21:19:45,NaN,NaN
4,142,www.newyorklawyersite.com,2006-03-18 08:02:58,NaN,NaN
5,142,www.newyorklawyersite.com,2006-03-18 08:03:09,NaN,NaN
6,142,westchester.gov,2006-03-20 03:55:57,1.0,http://www.westchestergov.com
7,142,space.comhttp,2006-03-24 20:51:24,NaN,NaN
8,142,dfdf,2006-03-24 22:23:07,NaN,NaN
9,142,dfdf,2006-03-24 22:23:14,NaN,NaN


In [30]:
import numpy as np

In [37]:
'.com' in 'rentdirect.com'

True

In [48]:
rem = ['.gov', '.com', '.edu', 'www', '.net', 'http', '.org']
for x in rem:
    data = data[[(x not in str(v)) for _,v in data.Query.iteritems()]]

In [49]:
data.shape

(2821032, 5)

In [50]:
data.iloc[1:200,:]

,AnonID,Query,QueryTime,ItemRank,ClickURL
9,142,dfdf,2006-03-24 22:23:14,NaN,NaN
13,142,207 ad2d 530,2006-04-08 01:31:04,NaN,NaN
14,142,207 ad2d 530,2006-04-08 01:31:14,1.0,http://www.courts.state.ny.us
22,142,merit release appearance,2006-04-22 23:51:18,NaN,NaN
27,217,lottery,2006-03-01 11:58:51,1.0,http://www.calottery.com
28,217,lottery,2006-03-01 11:58:51,1.0,http://www.calottery.com
30,217,susheme,2006-03-02 12:31:08,NaN,NaN
33,217,p; .; p;' p; ' ;' ;';,2006-03-09 12:09:27,NaN,NaN
34,217,p; .; p;' p; ' ;' ;';,2006-03-09 12:09:35,NaN,NaN
36,217,buddylis,2006-03-16 15:23:33,NaN,NaN
